In [53]:
# tokenizing

import datasets

dataset = datasets.load_dataset(
    "parquet", 
    data_files="./data/preprocessed_dataset.parquet", 
    split="train"
)
print(dataset)

Generating train split: 40474 examples [00:00, 97929.65 examples/s] 

Dataset({
    features: ['text'],
    num_rows: 40474
})


In [54]:
dataset = dataset.shard(num_shards=10,index=0)
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 4048
})


Each model has a specific tokenizer, otherwise the model work.

In [55]:
from transformers import AutoTokenizer
model_path = "upstage/SOLAR-10.7B-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_path,use_fast=False)

c:\Users\shahr\code\LLM_playground\.llmenv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [56]:
tokenizer.tokenize("What is my name ?")


['▁What', '▁is', '▁my', '▁name', '▁?']

In [57]:
def tokenization(example):
    #Tokenize
    tokens = tokenizer.tokenize(example['text'])

    #Convert token to ids
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Add <bos>, <eos> tokens to the front and back of token_ids
    # Shah-removed <bos> and <eos> due to error during tokenization
    #token_ids = [tokenizer.add_bos_token]+token_ids+[tokenizer.add_eos_token]
    
    example["input_ids"] = token_ids

    example['num_tokens'] = len(token_ids)
    return example

# Test the function
sentence_example = dataset[0]
sentence_example
e = tokenization(sentence_example)
print(e.keys())
print(f"Number of words in text: {len(e['text'])}")
print(f"Number of tokens in text: {e['num_tokens']}. Note that number of tokens<number of sentences.\
      Tokens are unique chars.")


dict_keys(['text', 'input_ids', 'num_tokens'])
Number of words in text: 2441
Number of tokens in text: 642. Note that number of tokens<number of sentences.      Tokens are unique chars.


## Tokenize all examples in the pretraining dataset

In [58]:
indices = list(range(1, 998)) + list(range(1000, len(dataset) ))
selected_dataset = dataset.select(indices)
print(len(selected_dataset))

4045


In [59]:
dataset_token = selected_dataset.map(tokenization,load_from_cache_file=False)
print(dataset_token)

Map:   0%|          | 0/4045 [00:00<?, ? examples/s]

Map: 100%|██████████| 4045/4045 [00:19<00:00, 205.81 examples/s]

Dataset({
    features: ['text', 'input_ids', 'num_tokens'],
    num_rows: 4045
})


In [60]:
sample = dataset[0]
sample_token = dataset_token[0]
print(sample)
print(sample_token)


{'text': "In 1793 Zaman Shah, a grandson of Ahmad Shah Durrani, won a brief war of succession to become ruler of Afghanistan. The support of Painda Khan, chief of the Baraksai branch of the Durrani tribe, was decisive in his victory. In the next fifty year., the brothers of Zaman shah and the sons of Painda Khan were to dominate the affairs of Afghanistan. The Durrani tribe was very large with several branches and numerous clans. 1 Abmad Shah and his successors belonged to the Sadozai clan, but other clans, such as the Mohammedzai of Painda Khan, were larger and more powerful and this situation caused many problems.\nMahmud had revolted unsuccessfully several times with Persian backing, but now with Fateh Khan's help he was able to defeat Zaman who was captured and blinded. Mahmud's position was insecure however. Persian invasions threatened, the tribes were discontented, and another brother of Zaman, Shuja-ul-Mulk, was in arms against him. In 1803 Shuja succeeded in toppling Mahmud af